# FastAPI CRUD Lab

In this lab, you'll build a complete CRUD (Create, Read, Update, Delete) API using FastAPI. By the end, you'll have a working API for managing a list of tasks.

**What you'll learn:**
- Setting up a FastAPI project
- Creating API endpoints
- Handling request data with Pydantic models
- Implementing CRUD operations
- Testing your API

**Prerequisites:**
- Python 3.10+ installed
- Basic Python knowledge

**Time:** ~45 minutes

---

## Part 1: Setup

First, let's install FastAPI and Uvicorn (the server that runs FastAPI).

```bash
pip install fastapi uvicorn
```

Create a new file called `main.py` in your project folder.

---

## Part 2: Hello World

Let's start with the simplest possible API.

### Create `main.py`:

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello, World!"}
```

### Run the server:

```bash
uvicorn main:app --reload
```

- `main` — the file name (main.py)
- `app` — the FastAPI instance
- `--reload` — auto-restart when code changes (development only)

### Test it:

Open your browser to:
- http://localhost:8000 — Your API response
- http://localhost:8000/docs — Interactive documentation (Swagger UI)
- http://localhost:8000/redoc — Alternative documentation

The automatic docs are one of FastAPI's best features!

---

## Part 3: Understanding the Basics

### Decorators Define Endpoints

```python
@app.get("/")      # Responds to GET requests at /
@app.post("/")     # Responds to POST requests at /
@app.put("/")      # Responds to PUT requests at /
@app.delete("/")   # Responds to DELETE requests at /
```

### Path Parameters

```python
@app.get("/users/{user_id}")
def get_user(user_id: int):
    return {"user_id": user_id}
```

The `{user_id}` in the path becomes a function parameter. The type hint `int` tells FastAPI to:
1. Validate that it's a number
2. Convert it from string to int

### Query Parameters

```python
@app.get("/users")
def list_users(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}
```

Called with: `/users?skip=20&limit=50`

Parameters with default values become optional query parameters.

---

## Part 4: Pydantic Models

Pydantic models define the shape of your data. FastAPI uses them for:
- Request body validation
- Response serialization
- Automatic documentation

### Define a Model

```python
from pydantic import BaseModel
from typing import Optional

class Task(BaseModel):
    title: str
    description: Optional[str] = None
    completed: bool = False
```

This model says:
- `title` — Required string
- `description` — Optional string (defaults to None)
- `completed` — Boolean (defaults to False)

### Use in an Endpoint

```python
@app.post("/tasks")
def create_task(task: Task):
    return task
```

FastAPI automatically:
- Parses the JSON request body
- Validates it against the Task model
- Returns a 422 error if validation fails

---

## Part 5: Building the Task API

Now let's build a complete CRUD API for tasks. Update your `main.py`:

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

app = FastAPI(
    title="Task API",
    description="A simple CRUD API for managing tasks",
    version="1.0.0"
)

# --- Models ---

class TaskCreate(BaseModel):
    """Model for creating a new task"""
    title: str
    description: Optional[str] = None

class TaskUpdate(BaseModel):
    """Model for updating a task"""
    title: Optional[str] = None
    description: Optional[str] = None
    completed: Optional[bool] = None

class Task(BaseModel):
    """Model for a task response"""
    id: int
    title: str
    description: Optional[str] = None
    completed: bool = False

# --- In-Memory Database ---

tasks_db: dict[int, dict] = {}
next_id = 1

# --- Endpoints ---

@app.get("/")
def read_root():
    return {"message": "Welcome to the Task API", "docs": "/docs"}
```

We have:
- Separate models for create, update, and response
- An in-memory "database" (dictionary)
- A counter for generating IDs

---

## Part 6: CREATE — Add a Task

Add the POST endpoint to create new tasks:

```python
@app.post("/tasks", response_model=Task, status_code=201)
def create_task(task: TaskCreate):
    """Create a new task"""
    global next_id
    
    new_task = {
        "id": next_id,
        "title": task.title,
        "description": task.description,
        "completed": False
    }
    
    tasks_db[next_id] = new_task
    next_id += 1
    
    return new_task
```

**Key points:**
- `response_model=Task` — Tells FastAPI what the response looks like (for docs)
- `status_code=201` — Returns 201 Created instead of 200
- We generate an ID and add the task to our "database"

### Test it:

```bash
curl -X POST http://localhost:8000/tasks \
  -H "Content-Type: application/json" \
  -d '{"title": "Learn FastAPI", "description": "Complete the CRUD lab"}'
```

Or use the interactive docs at `/docs`.

---

## Part 7: READ — Get Tasks

Add endpoints to retrieve tasks:

```python
@app.get("/tasks", response_model=list[Task])
def list_tasks(completed: Optional[bool] = None):
    """Get all tasks, optionally filtered by completion status"""
    tasks = list(tasks_db.values())
    
    if completed is not None:
        tasks = [t for t in tasks if t["completed"] == completed]
    
    return tasks


@app.get("/tasks/{task_id}", response_model=Task)
def get_task(task_id: int):
    """Get a specific task by ID"""
    if task_id not in tasks_db:
        raise HTTPException(status_code=404, detail="Task not found")
    
    return tasks_db[task_id]
```

**Key points:**
- `list[Task]` — Returns a list of tasks
- Optional query parameter for filtering
- `HTTPException` for error handling with proper status codes

### Test it:

```bash
# Get all tasks
curl http://localhost:8000/tasks

# Get only completed tasks
curl http://localhost:8000/tasks?completed=true

# Get a specific task
curl http://localhost:8000/tasks/1

# Try a non-existent task (should return 404)
curl http://localhost:8000/tasks/999
```

---

## Part 8: UPDATE — Modify a Task

Add the PATCH endpoint to update tasks:

```python
@app.patch("/tasks/{task_id}", response_model=Task)
def update_task(task_id: int, task_update: TaskUpdate):
    """Update a task (partial update)"""
    if task_id not in tasks_db:
        raise HTTPException(status_code=404, detail="Task not found")
    
    task = tasks_db[task_id]
    
    # Update only the fields that were provided
    if task_update.title is not None:
        task["title"] = task_update.title
    if task_update.description is not None:
        task["description"] = task_update.description
    if task_update.completed is not None:
        task["completed"] = task_update.completed
    
    return task
```

**Why PATCH?**
- PATCH allows partial updates (only send fields you want to change)
- PUT would require sending the entire object

### Test it:

```bash
# Mark task as completed
curl -X PATCH http://localhost:8000/tasks/1 \
  -H "Content-Type: application/json" \
  -d '{"completed": true}'

# Update title only
curl -X PATCH http://localhost:8000/tasks/1 \
  -H "Content-Type: application/json" \
  -d '{"title": "Learn FastAPI (done!)"}'
```

---

## Part 9: DELETE — Remove a Task

Add the DELETE endpoint:

```python
@app.delete("/tasks/{task_id}", status_code=204)
def delete_task(task_id: int):
    """Delete a task"""
    if task_id not in tasks_db:
        raise HTTPException(status_code=404, detail="Task not found")
    
    del tasks_db[task_id]
    
    return None  # 204 No Content
```

**Key points:**
- `status_code=204` — Standard for successful deletion
- Returns nothing (No Content)

### Test it:

```bash
# Delete a task
curl -X DELETE http://localhost:8000/tasks/1

# Verify it's gone
curl http://localhost:8000/tasks/1  # Should return 404
```

---

## Part 10: Complete Code

Here's the complete `main.py`:

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

app = FastAPI(
    title="Task API",
    description="A simple CRUD API for managing tasks",
    version="1.0.0"
)

# --- Models ---

class TaskCreate(BaseModel):
    title: str
    description: Optional[str] = None

class TaskUpdate(BaseModel):
    title: Optional[str] = None
    description: Optional[str] = None
    completed: Optional[bool] = None

class Task(BaseModel):
    id: int
    title: str
    description: Optional[str] = None
    completed: bool = False

# --- In-Memory Database ---

tasks_db: dict[int, dict] = {}
next_id = 1

# --- Endpoints ---

@app.get("/")
def read_root():
    return {"message": "Welcome to the Task API", "docs": "/docs"}

@app.post("/tasks", response_model=Task, status_code=201)
def create_task(task: TaskCreate):
    global next_id
    new_task = {
        "id": next_id,
        "title": task.title,
        "description": task.description,
        "completed": False
    }
    tasks_db[next_id] = new_task
    next_id += 1
    return new_task

@app.get("/tasks", response_model=list[Task])
def list_tasks(completed: Optional[bool] = None):
    tasks = list(tasks_db.values())
    if completed is not None:
        tasks = [t for t in tasks if t["completed"] == completed]
    return tasks

@app.get("/tasks/{task_id}", response_model=Task)
def get_task(task_id: int):
    if task_id not in tasks_db:
        raise HTTPException(status_code=404, detail="Task not found")
    return tasks_db[task_id]

@app.patch("/tasks/{task_id}", response_model=Task)
def update_task(task_id: int, task_update: TaskUpdate):
    if task_id not in tasks_db:
        raise HTTPException(status_code=404, detail="Task not found")
    task = tasks_db[task_id]
    if task_update.title is not None:
        task["title"] = task_update.title
    if task_update.description is not None:
        task["description"] = task_update.description
    if task_update.completed is not None:
        task["completed"] = task_update.completed
    return task

@app.delete("/tasks/{task_id}", status_code=204)
def delete_task(task_id: int):
    if task_id not in tasks_db:
        raise HTTPException(status_code=404, detail="Task not found")
    del tasks_db[task_id]
    return None
```

---

## Summary

You've built a complete CRUD API! Here's what each endpoint does:

| Method | Endpoint | Action |
|--------|----------|--------|
| GET | `/` | Welcome message |
| POST | `/tasks` | Create a task |
| GET | `/tasks` | List all tasks |
| GET | `/tasks/{id}` | Get one task |
| PATCH | `/tasks/{id}` | Update a task |
| DELETE | `/tasks/{id}` | Delete a task |

### Key Concepts

- **FastAPI** — Modern Python web framework
- **Pydantic** — Data validation using Python type hints
- **Path parameters** — `{task_id}` in URLs
- **Query parameters** — `?completed=true`
- **HTTP status codes** — 200, 201, 204, 404
- **HTTPException** — Returning error responses

---

## Exercises

1. **Add a due date** — Add a `due_date` field to tasks (optional datetime)

2. **Add tags** — Allow tasks to have multiple tags (list of strings)

3. **Add search** — Add a `q` query parameter to search tasks by title

4. **Add pagination** — Add `skip` and `limit` parameters to the list endpoint

---

## Next Steps

Continue to the **[Auth Basics Lab](./auth_basics_lab.ipynb)** to learn about authentication and environment variables.